# 検定と精度（分散/標準偏差）のつながり

このノートは、**推定量の精度（分散/MSE）**と**検定統計量の標準化**を一つの流れで確認するための整理メモです。

- 推定：精度 = 分散（不偏なら MSE=Var）
- 検定：差を分散（標準誤差）で割って何σかを見る
- プーリング：帰無仮説の下で分散に入る母数を推定して差し込む


## 1. 推定の精度 = MSE と分散

推定量 $\hat\theta$ の良さは

$$\mathrm{MSE}(\hat\theta)=E[(\hat\theta-\theta)^2]=\mathrm{Var}(\hat\theta)+\mathrm{Bias}(\hat\theta)^2$$

不偏なら $\mathrm{Bias}=0$ で **MSE=Var**。
**不偏推定量同士なら分散が小さい方が精度が高い**、が基本です。


## 2. 標準化（検定統計量）の意味

検定でも推定でも、よく

$$Z=\frac{\hat\theta-\theta_0}{\mathrm{SE}(\hat\theta)},\quad \mathrm{SE}=\sqrt{\mathrm{Var}}$$

のように **標準誤差で割る**。意味は

- 分子：ズレ（信号）
- 分母：そのズレがどれくらい普通に出るか（ノイズの尺度）

→ 「何σズレたか」に変換して比較可能にする。


## 3. プーリング推定（2標本比率）の例

データ（20代のSNS利用の架空集計）：

|        | 利用している | 利用していない | 計 |
|:--|--:|--:|--:|
| 20代男 | 38 | 73 | 111 |
| 20代女 | 60 | 46 | 106 |
| 計 | 98 | 119 | 217 |

帰無仮説 $H_0: p_1=p_2=p$ の下では

$$D=\hat p_1-\hat p_2,\quad \mathrm{Var}(D)=p(1-p)\left(\frac1{n_1}+\frac1{n_2}\right)$$

$p$ が未知なので **プール推定量**

$$\hat p=\frac{x_1+x_2}{n_1+n_2}$$

を分母に入れて

$$Z=\frac{D}{\sqrt{\hat p(1-\hat p)\left(\frac1{n_1}+\frac1{n_2}\right)}}$$

を作る。**差を分散で割って無次元化する**のが核。


In [ ]:
import numpy as np

x1, n1 = 38, 111
x2, n2 = 60, 106
p1 = x1 / n1
p2 = x2 / n2
p_pool = (x1 + x2) / (n1 + n2)
D = p1 - p2
se = np.sqrt(p_pool * (1 - p_pool) * (1 / n1 + 1 / n2))
z = D / se
p1, p2, p_pool, D, se, z


## 4. 自己相関があると平均の分散はどう増えるか

AR(1) の平均 $\bar y_T$ は

$$\mathrm{Var}(\bar y_T)=\frac{1}{T^2}\sum_{i=1}^T\sum_{j=1}^T\mathrm{Cov}(y_i,y_j)$$

で、正の自己相関があると IID より分散が大きくなります。
ここではシミュレーションで **ACF** と **平均の分散比**を確認します。


In [ ]:
import matplotlib.pyplot as plt

rng = np.random.default_rng(0)

def simulate_ar1(alpha=0.5, sigma=1.0, T=400):
    y = np.zeros(T)
    eps = rng.normal(0, sigma, T)
    for t in range(1, T):
        y[t] = alpha * y[t - 1] + eps[t]
    return y

def acf(x, max_lag=20):
    x = x - x.mean()
    denom = np.dot(x, x)
    return np.array([np.dot(x[:-lag], x[lag:]) / denom for lag in range(1, max_lag + 1)])

T = 400
alpha = 0.6
y = simulate_ar1(alpha=alpha, T=T)
iid = rng.normal(0, 1.0, T)

lags = np.arange(1, 21)
acf_y = acf(y, 20)

var_mean_ar = np.var([simulate_ar1(alpha=alpha, T=T).mean() for _ in range(500)])
var_mean_iid = np.var([rng.normal(0, 1.0, T).mean() for _ in range(500)])

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].plot(y, color='steelblue')
axes[0].set_title('AR(1) 時系列例')
axes[0].set_xlabel('t')

axes[1].bar(lags, acf_y, color='teal')
axes[1].set_title('ACF (lag 1-20)')
axes[1].set_xlabel('lag')

plt.tight_layout()
plt.show()

var_mean_ar, var_mean_iid, var_mean_ar / var_mean_iid


## 5. まとめ

- **推定**：不偏なら精度 = 分散（MSE=Var）
- **検定**：$\hat\theta-\theta_0$ を $\sqrt{\mathrm{Var}}$ で割って「何σ」かを見る
- **プーリング**：分散中の未知母数を $H_0$ の形で推定して差し込む

推定と検定は **“分散で割る”という共通構造**で一本化できます。
